In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

import torch
from torch_geometric.nn import GAE
from torch_geometric.utils import is_undirected, to_undirected


from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from model.GCN_GRU import GCN_GRU, Decoder
from utils import *

In [28]:
shop_col = 'stonc_6_label'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

embedding_dim = 64
entity_dim = 8
epochs = 1000
learning_rate = 0.001
batch_size = 1000

weights_path = 'GCN_GRU_Link_Prediction'

In [3]:
data_path = './data'

chid_dict_file = 'sample/sample_50k_idx_map.npy'
cdtx_file = 'sample/sample_50k_cdtx.csv'
cust_file = 'preprocessed/df_cust_log_without_shop.csv'

chid_path = os.path.join(data_path, chid_dict_file)
cdtx_path = os.path.join(data_path, cdtx_file)
cust_path = os.path.join(data_path, cust_file)

In [4]:
df_cdtx, df_cust, n_users, n_shops = read_sample_files(cdtx_path,
                                                       cust_path,
                                                       chid_path,
                                                       shop_col)

Start reading cdtx file...
Finish reading cdtx file !
Start reading cust file...
Finish reading cust file !
Start maping encodeing...
Finish !!


In [10]:
list_months = sorted(df_cdtx.csmdt.unique())
ma = df_cdtx.groupby(['chid', 'csmdt']).objam.count().max()

ignore_cols = ['chid', 'data_dt']
category_cols = [f'category_{i+1}' for i in range(6)]
numeric_cols = list(set(df_cust.columns) - set(category_cols) - set(ignore_cols))

emb_dims = list(df_cust[category_cols].nunique())

input_dim = len(category_cols)*entity_dim + len(numeric_cols)

In [9]:
edge_dict = {}
edge_weights_dict = {}
for month in tqdm(list_months):
    edges = df_cdtx[df_cdtx.csmdt==month].groupby(['chid', shop_col]).objam.sum()
    edge_pairs = np.stack([np.array(i) for i in edges.index]).T
    edge_pairs = torch.LongTensor(edge_pairs)
    
    edge_weights = np.log(edges.values+1)
    edge_weights = torch.Tensor(edge_weights)
    
    if not is_undirected(edge_pairs):
        edge_pairs = torch.cat([edge_pairs, edge_pairs[[1,0],:]], -1)
        edge_weights = edge_weights.repeat(2)
    
    edge_dict[month] = edge_pairs
    edge_weights_dict[month] = edge_weights

100%|██████████| 25/25 [00:21<00:00,  1.19it/s]


In [11]:
temp_cdtx = df_cdtx.groupby(['csmdt', 'chid']).count()

df_cdtx_count = pd.DataFrame(list(map(list, temp_cdtx.index)), columns=['data_dt','chid' ])
df_cdtx_count['count'] = temp_cdtx.objam.values

df_y = pd.DataFrame({'chid':df_cust.chid, 'data_dt':df_cust.data_dt})
df_y = df_y.merge(df_cdtx_count,
                  how='left',
                  left_on=['chid', 'data_dt'],
                  right_on=['chid', 'data_dt']).fillna(0)

y_scaler = MinMaxScaler()
df_y['count'] = y_scaler.fit_transform(df_y[['count']])

In [12]:
model = GAE(GCN_GRU(input_dim, embedding_dim, entity_dim, emb_dims), Decoder(embedding_dim, 1)).to(device)

x_num = []
x_cat = []
y = []
for i in list_months:
    cust_num_features = df_cust[df_cust.data_dt==i][numeric_cols].to_numpy()
    cust_num_features = torch.Tensor(cust_num_features)
    cust_cat_features = df_cust[df_cust.data_dt==i][category_cols].to_numpy()
    cust_cat_features = torch.LongTensor(cust_cat_features)
    
    shop_num_features = torch.zeros(n_shops, cust_num_features.shape[1])
    shop_cat_features = torch.zeros(n_shops, cust_cat_features.shape[1]).long()
    
    temp_y = df_y[df_y.data_dt==i][['count']].to_numpy()
    x_num.append(torch.cat([cust_num_features, shop_num_features], 0).to(device))
    x_cat.append(torch.cat([cust_cat_features, shop_cat_features], 0).to(device))
    y.append(torch.Tensor(temp_y).to(device))
    
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
pos_edge_index = [i.to(device) for i in edge_dict.values()]
pos_edge_weigts = [i.to(device) for i in edge_weights_dict.values()]

In [29]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [13]:
model.encoder.load_state_dict(torch.load(weights_path))
model.eval()

criterion = torch.nn.MSELoss()

In [30]:
def train():
    model.train()
    train_output = np.array([])
    train_y = np.array([])
    for i in tqdm(range(10)):
        
        train_dataset = TensorDataset(y[i+12])
        train_loader = DataLoader(dataset=train_dataset, shuffle=False, batch_size=batch_size)
        
        for j, true_y in enumerate(train_loader):
            optimizer.zero_grad()
            z = model.encode(x_cat[i:i+12], x_num[i:i+12], pos_edge_index[i:i+12], pos_edge_weigts[i:i+12])
            output = model.decode(z[j*batch_size:(j+1)*batch_size])
            loss = criterion(output, true_y[0])
            train_output = np.concatenate([train_output, output.cpu().detach().numpy().reshape(-1)])
            train_y = np.concatenate([train_y, true_y[0].cpu().detach().numpy().reshape(-1)])
        
            loss.backward()
            optimizer.step()
        
            
    return loss/10, train_output, train_y

In [31]:
def test():
    model.eval()
    test_output = np.array([])
    test_y = np.array([])
    
    for i in range(10,12):

        with torch.no_grad():
            z = model.encode(x_cat[i:i+12], x_num[i:i+12], pos_edge_index[i:i+12], pos_edge_weigts[i:i+12])
            output = model.decode(z[:n_users])
            test_output = np.concatenate([test_output, output.cpu().detach().numpy().reshape(-1)])
            test_y = np.concatenate([test_y, y[i+12].cpu().detach().numpy().reshape(-1)])
            
    return test_output, test_y

In [27]:
for epoch in range(epochs):
    loss, train_output, train_y  = train()
    
    test_output, test_y = test()
    
    train_RMSE = mean_squared_error(train_output*ma, train_y*ma, squared=False)
    test_RMSE = mean_squared_error(test_output*ma, test_y*ma, squared=False)
    
    train_MAE = mean_absolute_error(train_output*ma, train_y*ma)
    test_MAE = mean_absolute_error(test_output*ma, test_y*ma)
    
    print(f'epoch:{epoch+1}\ntrain loss:{train_RMSE},test loss:{test_RMSE}\
    \ntrain MAE(mean):{train_MAE},test MAE(mean):{test_MAE}')
    

  0%|          | 0/10 [00:00<?, ?it/s]

epoch:1
train loss:11.648646227250044,test loss:16.946317691194288    
train MAE(mean):6.83003429597457,test MAE(mean):6.438409212848208


  0%|          | 0/10 [00:00<?, ?it/s]

epoch:2
train loss:10.34275565127834,test loss:16.431955521573624    
train MAE(mean):5.513244531307457,test MAE(mean):5.487134947807025


  0%|          | 0/10 [00:00<?, ?it/s]

...

epoch:29
train loss:8.612588503945538,test loss:15.57897537804162    
train MAE(mean):3.2702999647358935,test MAE(mean):3.5378094053335207


  0%|          | 0/10 [00:00<?, ?it/s]

epoch:30
train loss:8.602321795615607,test loss:15.575550999872226    
train MAE(mean):3.2524148576881755,test MAE(mean):3.5213822896759397


  0%|          | 0/10 [00:11<?, ?it/s]


KeyboardInterrupt: 

In [32]:
for epoch in range(epochs):
    loss, train_output, train_y  = train()
    
    test_output, test_y = test()
    
    train_RMSE = mean_squared_error(train_output*ma, train_y*ma, squared=False)
    test_RMSE = mean_squared_error(test_output*ma, test_y*ma, squared=False)
    
    train_MAE = mean_absolute_error(train_output*ma, train_y*ma)
    test_MAE = mean_absolute_error(test_output*ma, test_y*ma)
    
    print(f'epoch:{epoch+1}\ntrain loss:{train_RMSE},test loss:{test_RMSE}\
    \ntrain MAE(mean):{train_MAE},test MAE(mean):{test_MAE}')
    

  0%|          | 0/10 [00:00<?, ?it/s]

epoch:1
train loss:14.316886125658321,test loss:15.72129677653311    
train MAE(mean):5.711408838745013,test MAE(mean):3.7261303053996153


  0%|          | 0/10 [00:00<?, ?it/s]

epoch:2
train loss:8.684745085327988,test loss:15.645626224464534    
train MAE(mean):3.3524675252033584,test MAE(mean):3.921499982777694


  0%|          | 0/10 [00:00<?, ?it/s]

...

epoch:5
train loss:8.733743802418836,test loss:16.49743289297601    
train MAE(mean):3.4973764664376805,test MAE(mean):6.9940795610826


  0%|          | 0/10 [00:00<?, ?it/s]

epoch:6
train loss:8.713782531694132,test loss:15.627515791361136    
train MAE(mean):3.460601090174191,test MAE(mean):3.4072855171816796


 70%|███████   | 7/10 [01:30<00:38, 12.93s/it]


KeyboardInterrupt: 